# Text to SQL

In [10]:
!curl "https://api.mockaroo.com/api/bd11c330?count=1000&key=93f9e9d0" > "employee.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 88798    0 88798    0     0  82786      0 --:--:--  0:00:01 --:--:-- 82756


In [11]:
!curl "https://api.mockaroo.com/api/280de6d0?count=1000&key=93f9e9d0" > "department.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27820    0 27820    0     0  17141      0 --:--:--  0:00:01 --:--:-- 17141


In [12]:
import sqlite3
import pandas as pd
import os

In [20]:
#SQL schemas
employees_schema = """
CREATE TABLE IF NOT EXISTS employee (
employee_id INTEGER PRIMARY KEY,
first_name TEXT,
last_name TEXT,
email TEXT,
salary INTEGER,
hire_date TEXT,
department_name TEXT,
age INTEGER,
job_title TEXT
);
"""

departments_schema = """
CREATE TABLE IF NOT EXISTS department (
department_id INTEGER PRIMARY KEY,
employee_id INTEGER,
department_name TEXT,
department_size INTEGER,
department_location TEXT,
FOREIGN KEY (employee_id) REFERENCES employee (employee_id)
);
"""

In [14]:
# Connect to SQLite database
conn = sqlite3.connect('employee.db')
cursor = conn.cursor()

# Create the table if it doesn't exist
cursor.execute(employees_schema)

# Load data from CSV into pandas DataFrame
employees_df = pd.read_csv('employee.csv')

# Write the data from the DataFrame to the SQLite table
employees_df.to_sql('employee', conn, if_exists='replace', index=False)

# Verify data insertion
cursor.execute("SELECT COUNT(*) FROM employee;")
count = cursor.fetchone()[0]
print(f"Number of rows in employee table: {count}")

# Close the connection
conn.close()

Number of rows in employee table: 1000


In [21]:
conn = sqlite3.connect('department.db')
cursor = conn.cursor()

# Create the table if it doesn't exist
cursor.execute(departments_schema)

# Load data from CSV into pandas DataFrame
departments_df = pd.read_csv('department.csv')

# Write the data from the DataFrame to the SQLite table
departments_df.to_sql('department', conn, if_exists='replace', index=False)

# Verify data insertion
cursor.execute("SELECT COUNT(*) FROM department;")
count = cursor.fetchone()[0]
print(f"Number of rows in department table: {count}")

# Close the connection
conn.close()

Number of rows in department table: 1000


In [28]:
import sqlite3
import os

# Define database file names
company_db = 'company.db'
employee_db = 'employee.db'
department_db = 'department.db'

# Delete existing database files to ensure a clean start and avoid locks
if os.path.exists(company_db):
    os.remove(company_db)
if os.path.exists(employee_db):
    os.remove(employee_db)
if os.path.exists(department_db):
    os.remove(department_db)

# Re-create employee.db from CSV
conn_emp = sqlite3.connect(employee_db)
cursor_emp = conn_emp.cursor()
cursor_emp.execute(employees_schema)
employees_df = pd.read_csv('employee.csv')
employees_df.to_sql('employee', conn_emp, if_exists='replace', index=False)
conn_emp.close()

# Re-create department.db from CSV
conn_dept = sqlite3.connect(department_db)
cursor_dept = conn_dept.cursor()
cursor_dept.execute(departments_schema)
departments_df = pd.read_csv('department.csv')
departments_df.to_sql('department', conn_dept, if_exists='replace', index=False)
conn_dept.close()

# Connect to the new unified database
conn = sqlite3.connect(company_db)
cursor = conn.cursor()

# Enforce foreign keys
cursor.execute("PRAGMA foreign_keys = ON;")

# Attach existing databases
cursor.execute(f"ATTACH DATABASE '{employee_db}' AS emp_db;")
cursor.execute(f"ATTACH DATABASE '{department_db}' AS dept_db;")

# Create employee table in company.db
cursor.execute(employees_schema)

# Create department table in company.db
cursor.execute(departments_schema)

# Copy data from employee.db → company.db
cursor.execute("""
INSERT INTO employee (employee_id, first_name, last_name, email, salary, hire_date, department_name, age, job_title)
SELECT employee_id, first_name, last_name, email, salary, hire_date, department, age, job_title
FROM emp_db.employee;
""")

# Copy data from department.db → company.db
cursor.execute("""
INSERT INTO department (department_id, employee_id, department_name, department_size, department_location)
SELECT department_id, employee_id, department_name, department_size, department_location
FROM dept_db.department;
""")

# Verify counts
cursor.execute("SELECT COUNT(*) FROM employee;")
print("Employees:", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM department;")
print("Departments:", cursor.fetchone()[0])

# Detach databases
cursor.execute("DETACH DATABASE emp_db;")
cursor.execute("DETACH DATABASE dept_db;")

conn.commit()
conn.close()

Employees: 1000
Departments: 1000


In [29]:
employees_df.head()

,employee_id,first_name,last_name,age,email,job_title,department,salary,hire_date
0,1,Persis,Dubose,60,pdubose0@google.de,Payment Adjustment Coordinator,HR,49246.11,1/31/2018
1,2,Meagan,Schuster,23,mschuster1@washington.edu,Biostatistician IV,Finance,36055.82,2/22/2014
2,3,Christoffer,Exroll,52,cexroll2@marriott.com,Cost Accountant,Sales,84528.05,10/23/2019
3,4,Brook,Pllu,31,bpllu3@altervista.org,Sales Associate,Finance,61206.01,10/17/2013
4,5,Garner,Drynan,58,gdrynan4@photobucket.com,Geologist I,Marketing,146070.67,2/6/2017


In [30]:
departments_df.head()

,department_name,department_size,department_location,department_id,employee_id
0,IT,315,PO Box 61468,1,69
1,HR,77,Apt 1937,2,208
2,Finance,241,9th Floor,3,195
3,Sales,89,Suite 90,4,5
4,Finance,38,Room 1076,5,835


In [16]:
!pip install google-genai

In [18]:
from google import genai
from google.colab import userdata
genai_client = genai.Client(api_key=userdata.get('google_api_key'))

In [31]:
prompt = """
### ROLE

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### CONTEXT

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's employee database using natural language. The database dialect is always SQLite. Your responses will be executed directly on the database.

The database consists of the following tables in the single database file `company.db`:

Employee table

CREATE TABLE IF NOT EXISTS employee (
  employee_id INTEGER PRIMARY KEY,
  first_name TEXT,
  last_name TEXT,
  email TEXT,
  salary INTEGER,
  hire_date TEXT,
  department_name TEXT,
  age INTEGER,
  job_title TEXT
);

Department table

CREATE TABLE IF NOT EXISTS department (
  department_id INTEGER PRIMARY KEY,
  employee_id INTEGER,
  department_name TEXT,
  department_size INTEGER,
  department_location TEXT,
  FOREIGN KEY (employee_id) REFERENCES employee (employee_id)
);

-----

### TASK

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1. Analyze the User's Query: Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like SUM, COUNT, AVG), and ordering they are asking for.
2. Map to the Schema: Map the entities from the user's query to the appropriate columns in the employee or department tables. Use only the columns provided in the schema.
3. Construct the SQLite Query: Write a clean and efficient SELECT statement that is syntactically correct for SQLite. Ensure all column names are accurate.
4. Handle Ambiguity: If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### CONSTRAINTS

- Read-Only Operations: You must only generate SELECT queries. Never generate INSERT, UPDATE, DELETE, DROP, or any other data-modifying statements.
- Adhere Strictly to Schema: Only use the employee and department tables and their defined columns. Do not invent or assume the existence of any other tables or columns.
- No Explanations: Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.
- Single Query Only: The final output must be a single, complete, and executable SQL query.
- Handle Impossibility: If a request is impossible to fulfill with the given schema (for example, "Which employee made the most sales?"), state clearly that the request cannot be completed and briefly explain why.

-----

### EXAMPLES

Example 1: Simple Lookup
User Query: "Show me all employees in the IT department"
Expected Output:
{
  "status": "success",
  "response": "SELECT * FROM employee WHERE department_name = 'IT';"
}

Example 2: Aggregation and Sorting
User Query: "What are the top 3 highest-paid employees?"
Expected Output:
{
  "status": "success",
  "response": "SELECT first_name, last_name, salary FROM employee ORDER BY salary DESC LIMIT 3;"
}

Example 3: Ambiguous Query
User Query: "Show me recent employees"
Expected Output:
{
  "status": "clarification_needed",
  "response": "Could you please define what 'recent' means? For example, 'hired in the last 30 days', 'this year', or 'since January 2025'."
}

Example 4: Impossible Query
User Query: "Which employee made the most sales?"
Expected Output:
{
  "status": "error",
  "response": "I cannot answer this question as the database does not contain information about sales."
}

-----

### OUTPUT FORMAT

Your final response must be a single JSON object with two keys:

1. "status": A string with one of three possible values: "success", "clarification_needed", or "error".
2. "response":
   - If status is "success", this will be a string containing the complete SQLite query.
   - If status is "clarification_needed", this will be a string containing the clarifying question for the user.
   - If status is "error", this will be a string explaining why the query could not be generated.
"""


In [32]:
import json
def get_sql_query(genai_client, prompt, user_query):


  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response)

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json', '').replace('```', ''))

  return output

In [35]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='company.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [33]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  return output

In [36]:
text2sql(genai_client, prompt, "Show me the number of employees in each department")

Input Token Count: 1063
Thoughts Token Count: 32
Output Token Count: 47
Total Token Count: 1142

Executing query on 'company.db':
SELECT department_name, COUNT(employee_id) AS number_of_employees FROM employee GROUP BY department_name;
Query executed successfully.


,department_name,number_of_employees
0,Finance,207
1,HR,201
2,IT,213
3,Marketing,181
4,Sales,198


In [37]:
text2sql(genai_client, prompt, "List all employees who live in 'Amsterdam'")




Input Token Count: 1063
Thoughts Token Count: 165
Output Token Count: 41
Total Token Count: 1269


{'status': 'error',
 'response': "I cannot answer this question as the database does not contain information about employees' city of residence."}

In [38]:
text2sql(genai_client, prompt, "What is the average salary of employees in the 'HR' department?")

Input Token Count: 1068
Thoughts Token Count: 72
Output Token Count: 37
Total Token Count: 1177

Executing query on 'company.db':
SELECT AVG(salary) FROM employee WHERE department_name = 'HR';
Query executed successfully.


,AVG(salary)
0,92929.908905


In [39]:

text2sql(genai_client, prompt, "Show employees older than 40 who work in the 'Finance' department")

Input Token Count: 1069
Thoughts Token Count: 66
Output Token Count: 40
Total Token Count: 1175

Executing query on 'company.db':
SELECT * FROM employee WHERE age > 40 AND department_name = 'Finance';
Query executed successfully.


,employee_id,first_name,last_name,email,salary,hire_date,department_name,age,job_title
0,9,Pebrook,Creeghan,pcreeghan8@bandcamp.com,44258.56,7/6/2014,Finance,58,Engineer I
1,27,Alanna,Beaulieu,abeaulieuq@senate.gov,106670.90,11/8/2015,Finance,47,Associate Professor
2,29,Cherry,Carleton,ccarletons@nba.com,84617.20,3/3/2021,Finance,60,Accounting Assistant I
3,35,Arlin,Hasling,ahaslingy@nature.com,110429.40,8/10/2018,Finance,52,Staff Scientist
4,50,Dale,Windeatt,dwindeatt1d@jigsy.com,130832.31,9/7/2022,Finance,52,Paralegal
...,...,...,...,...,...,...,...,...,...
101,926,Franni,Van der Kruijs,fvanderkruijspp@drupal.org,62486.57,11/3/2013,Finance,50,Programmer I
102,930,Shep,Stanlick,sstanlickpt@163.com,88305.19,6/21/2013,Finance,46,Financial Analyst
103,959,Dyanne,Costanza,dcostanzaqm@microsoft.com,135850.15,8/28/2018,Finance,47,Senior Editor
104,963,Vinnie,Joslow,vjoslowqq@google.it,118106.61,11/6/2012,Finance,56,Account Executive


In [40]:

text2sql(genai_client, prompt, "Show the first name, last name, and department location for employees in the 'IT' department")

Input Token Count: 1073
Thoughts Token Count: 2794
Output Token Count: 70
Total Token Count: 3937

Executing query on 'company.db':
SELECT e.first_name, e.last_name, d.department_location FROM employee AS e JOIN department AS d ON e.department_name = d.department_name WHERE e.department_name = 'IT';
Query executed successfully.


,first_name,last_name,department_location
0,Van,Grandisson,10th Floor
1,Van,Grandisson,10th Floor
2,Van,Grandisson,11th Floor
3,Van,Grandisson,11th Floor
4,Van,Grandisson,12th Floor
...,...,...,...
39613,Cesya,Dashper,Suite 89
39614,Cesya,Dashper,Suite 9
39615,Cesya,Dashper,Suite 90
39616,Cesya,Dashper,Suite 94
